## Setup

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
PROJECT = 'pokoyakazan-test-01'
BUCKET = 'pokoyakazan-test-01'
REGION = 'us-central1'

os.environ['BUCKET'] = BUCKET

## Exploration using Spark SQL of Dataproc
- 以下の分析はBQでもできるが今回はSpark SQLをGCSから読み込んで使ってみる

### Start a SparkSession

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Bays classification using Spark") \
    .getOrCreate()

### flightsデータ読み込み

#### Sparkで読み込むためのSchemaを定義

In [ ]:
# Set up schema to read in the CSV files on GCS
from pyspark.sql.types import StringType, FloatType, StructType, StructField

header = 'FL_DATE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,DISTANCE,DEP_AIRPORT_LAT,DEP_AIRPORT_LON,DEP_AIRPORT_TZOFFSET,ARR_AIRPORT_LAT,ARR_AIRPORT_LON,ARR_AIRPORT_TZOFFSET,EVENT,NOTIFY_TIME'

def get_structfield(colname):
  if colname in ['ARR_DELAY', 'DEP_DELAY', 'DISTANCE']:
    return StructField(colname, FloatType(), True)
  else:
    return StructField(colname, StringType(), True)
  
schema = StructType([get_structfield(colname) for colname in header.split(',')])
print(schema)

-> ARR_DELAY', 'DEP_DELAY', 'DISTANCE'以外はStringTypeになっている

#### SchemaとGCS上のcsvのPathを指定してflightsデータを読み込む

In [ ]:
# Create a table definition (this is done lazily; the files won't be read until we issue a query)
#inputs = 'gs://{}/flights/tzcorr/all_flights-00000-*'.format(BUCKET)
inputs = 'gs://{}/flights/tzcorr/all_flights-*'.format(BUCKET)
flights_csv = spark.read\
            .schema(schema)\
            .csv(inputs)

#### 読み込んだデータからTempView作成

In [ ]:
flights_csv.createOrReplaceTempView('flights_tmp')

In [ ]:
# check
results = spark.sql('SELECT COUNT(*) FROM flights_tmp WHERE dep_delay > -20 AND distance < 2000')
results.show()
results = spark.sql('SELECT * FROM flights_tmp WHERE dep_delay > -20 AND distance < 2000')
results.head(1)

### train_dayデータ読み込み

trainday.csvは小さいデータなのでspark側にカラムごとの型を推測(inferSchema)させる
- また、最初の一行が絡む名なのでheaderをtrue

In [ ]:
inputs = 'gs://{}/flights/trainday.csv'.format(BUCKET)
traindays = spark.read \
  .option("header", "true")\
  .option("inferSchema", "true")\
  .csv(inputs)
traindays.createOrReplaceTempView('traindays_tmp')

In [ ]:
# check
results = spark.sql('select * from traindays_tmp')
results.head(5)

### Make train data

In [ ]:
statement = """
select
 f.FL_DATE AS date,
 distance,
 dep_delay
from flights_tmp f
join traindays_tmp t
on f.FL_DATE==t.FL_DATE
where t.is_train_day
and f.dep_delay is not null
order by f.dep_delay desc
"""
train_flights = spark.sql(statement)

#### トレーニングデータの統計情報確認

"件数"、"フライト距離、出発遅延それぞれの最大値、平均、最小値"など

In [ ]:
# plotするためにsparkデータフレームをpandasデータフレームに変換
df = train_flights[(train_flights['distance'] < 2000)
                   & (train_flights['dep_delay'] > -20)
                   & (train_flights['dep_delay'] < 30)]
df.describe().show()

Hexbin plot で分布確認
- フライト距離は250~500に集中
- 出発遅延時間は-5~0に集中

In [ ]:
# dfから2%をサンプリングする
pdf = df.sample(False, 0.02, 20).toPandas() # to 100,000 rows approx on complete dataset
g = sns.jointplot(pdf['distance'], pdf['dep_delay'], kind="hex",
                  size=10, joint_kws={'gridsize':20})

## ヒストブラムイコライゼーション

### データを均一に分ける境界値を見つける
近似分位法を行ってデータ数が同じになるように間隔をきる
- APPROX_QUANTILES
  - BigQueryで近似分位法を行う関数
- **approxQuantile** -> **今回はこっちを使用**
  - sparkで近似分位法を行う関数

In [ ]:
# フライト距離の境界値
distthresh = train_flights.approxQuantile('distance', list(np.arange(0, 1.0, 0.1)), 0.02)
distthresh # 最初の値は最小値

In [ ]:
# 出発遅延の境界値
delaythresh = train_flights.approxQuantile('dep_delay', list(np.arange(0, 1.0, 0.1)), 0.05)
delaythresh # 最初の値は最小値

すべてのフライトのおよそ1/10は1.0分から3.0分の出発遅延時間を持つ

-> **出発遅延時間が17分以上 かつ フライト距離が2419マイル以上のフライトは全体の1/100**

In [ ]:
results = spark.sql('SELECT COUNT(*) FROM flights_tmp WHERE dep_delay >= 17 AND distance >= 2419')
results.show()

## 各セルにおいて時間通りに到着するフライトが70%以上かどうかを計算
これ以降の計算はSparkでも可能だが、ここではPigを使う